In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

tol = 0.01
eps = 0.01

Pre-processing, Data Cleaning, Feature Engineering

In [ ]:
dataDf = pd.read_excel("data.xlsx", dtype=object)
columnNames = dataDf.columns

dataDf.dropna(inplace=True)
dataDf['GENDER'] = dataDf['GENDER'].replace(["Male", "Female"], [1,2])
for i in range(4, 43-9):
    dataDf = dataDf.astype({dataDf.columns[i]: str})

for i in range(len(dataDf)):
    row = dataDf.iloc[i]
    for c in range(4, 18):
        row[columnNames[c]] = row[columnNames[c]].strip(".")
        row[columnNames[c]] = row[columnNames[c]].split('.')
    dataDf.iloc[i] = row
for i in range(len(dataDf)):
    for c in range(4, 18):
        dataDf = dataDf.explode(columnNames[c])

dropList = ['Questioinnaire number ', 'AGE', 'HOME LANGUAGE']
dataDf = dataDf.drop(columns=dropList)

dataDf['PASSED'] = 0
dataDf.loc[dataDf['FNL ILS'] >= 50, 'PASSED'] = 1

columns = dataDf.columns

In [ ]:
miscDf = dataDf[columns[:7]]
miscDf['PASSED'] = dataDf['PASSED']
miscDf = miscDf.drop_duplicates()
#miscDf

In [ ]:
dataDf = pd.read_excel("data.xlsx", dtype=object)
dataDf['PASSED'] = 0
dataDf.loc[dataDf['FNL ILS'] >= 50, 'PASSED'] = 1; columns = dataDf.columns

In [ ]:
schoolExpDf = dataDf[columns[13:18]]
schoolExpDf['PASSED'] = dataDf['PASSED']
#schoolExpDf = schoolExpDf.dropna()
schoolExpCols = schoolExpDf.columns
for i in range(len(schoolExpDf)):
    for c in schoolExpCols:
        schoolExpDf[c].iloc[i] = str(schoolExpDf[c].iloc[i]).strip(".")
        schoolExpDf[c].iloc[i] = str(schoolExpDf[c].iloc[i]).split('.')
for i in range(len(schoolExpDf)):
    for c in schoolExpCols:
        schoolExpDf = schoolExpDf.explode(c)
#schoolExpDf

In [ ]:
familyDf = dataDf[columns[30:34]]
familyDf[columns[10]] = dataDf[columns[10]]
familyDf[columns[11]] = dataDf[columns[11]]
familyDf['PASSED'] = dataDf['PASSED']
#familyDf = familyDf.dropna()
familyCols = familyDf.columns
for i in range(len(familyDf)):
    for c in familyCols:
        familyDf[c].iloc[i] = str(familyDf[c].iloc[i]).strip(".")
        familyDf[c].iloc[i] = str(familyDf[c].iloc[i]).split('.')
for i in range(len(familyDf)):
    for c in familyCols:
        familyDf = familyDf.explode(c)
#familyDf

In [ ]:
peerDf = dataDf[columns[22:26]]
peerDf['PASSED'] = dataDf['PASSED']
#peerDf = peerDf.dropna()
peerCols = peerDf.columns
for i in range(len(peerDf)):
    for c in peerCols:
        peerDf[c].iloc[i] = str(peerDf[c].iloc[i]).strip(".")
        peerDf[c].iloc[i] = str(peerDf[c].iloc[i]).split('.')
for i in range(len(peerDf)):
    for c in peerCols:
        peerDf = peerDf.explode(c)
#peerDf

In [ ]:
schoolBackDf = dataDf[columns[26:30]]
schoolBackDf['PASSED'] = dataDf['PASSED']
#schoolBackDf = schoolBackDf.dropna()
schoolBackCols = schoolBackDf.columns
for i in range(len(schoolBackDf)):
    for c in schoolBackCols:
        schoolBackDf[c].iloc[i] = str(schoolBackDf[c].iloc[i]).strip(".")
        schoolBackDf[c].iloc[i] = str(schoolBackDf[c].iloc[i]).split('.')
for i in range(len(schoolBackDf)):
    for c in schoolBackCols:
        schoolBackDf = schoolBackDf.explode(c)
#schoolBackDf

In [ ]:
selfDf = dataDf[columns[30:34]]
selfDf['PASSED'] = dataDf['PASSED']
#selfDf = selfDf.dropna()
selfCols = selfDf.columns
for i in range(len(selfDf)):
    for c in selfCols:
        selfDf[c].iloc[i] = str(selfDf[c].iloc[i]).strip(".")
        selfDf[c].iloc[i] = str(selfDf[c].iloc[i]).split('.')
for i in range(len(selfDf)):
    for c in selfCols:
        selfDf = selfDf.explode(c)
#selfDf

In [ ]:
def gaussianKernel(x, y, sigma=1):
    if np.ndim(x) == 1 and np.ndim(y) == 1:
        result = np.exp(- (np.linalg.norm(x - y, 2)) ** 2 / (2 * sigma ** 2))
    elif (np.ndim(x) > 1 and np.ndim(y) == 1) or (np.ndim(x) == 1 and np.ndim(y) > 1):
        result = np.exp(- (np.linalg.norm(x - y, 2, axis=1) ** 2) / (2 * sigma ** 2))
    elif np.ndim(x) > 1 and np.ndim(y) > 1:
        result = np.exp(- (np.linalg.norm(x[:, np.newaxis] - y[np.newaxis, :], 2, axis=2) ** 2) / (2 * sigma ** 2))
    return result

In [ ]:
def linearKernel(X1,X2):
    return np.inner(X1,X2)

In [ ]:
def objectiveFunction(alphas, y, kernel, X):
    return np.sum(alphas) - 0.5 * np.sum((y[:, None] * y[None, :]) * kernel(X, X) * (alphas[:, None] * alphas[None, :]))

In [ ]:
def decisionFunction(alphas, y, kernel, X_train, x_test, b):
    result = (alphas * y) @ kernel(X_train, x_test) - b
    return result

In [ ]:
class SMOModel:
    def __init__(self,X,Y,C,b,kernel):
        self.m = Y.shape[0]
        self.y = Y
        self.X = X
        self.C = C
        self.b = b
        self.kernel = kernel
        self.alphas = np.zeros(self.m)
        self.errors = decisionFunction(self.alphas, self.y, self.kernel,self.X, self.X, self.b) - self.y

In [ ]:
def takeStep(i1, i2, model):
    
    if i1 == i2:
        return 0, model
    
    alph1 = model.alphas[i1]
    alph2 = model.alphas[i2]
    y1 = model.y[i1]
    y2 = model.y[i2]
    E1 = model.errors[i1]
    E2 = model.errors[i2]
    
    s = y1*y2
    if y1 == y2:
        L = max(0 , alph1 + alph2 - model.C)
        H = min(model.C, alph1 + alph2)
    else:
        L = max(0, alph2 - alph1)
        H = min(model.C, model.C + alph2 - alph1)
    if L == H:
        return 0, model
    
    k11 = model.kernel(model.X[i1],model.X[i1])
    k12 = model.kernel(model.X[i1],model.X[i2])
    k22 = model.kernel(model.X[i2],model.X[i2])
    eta = k11 + k22 - 2*k12
    
    if eta > 0:
        a2 = alph2 + (y2*(E1 -E2))/eta
        if a2 < L:
            a2 = L
        elif a2 > H:
            a2 = H
    else:
        alphas_adj = model.alphas.copy()
        alphas_adj[i2] = L
        Lobj = objectiveFunction(alphas_adj, model.y, model.kernel, model.X)
        alphas_adj[i2] = H
        Hobj = objectiveFunction(alphas_adj, model.y, model.kernel, model.X)
        
        if Lobj < (Hobj - eps):
            a2 = L
        elif Lobj > (Hobj + eps):
            a2 = H
        else:
            a2 = alph2
    
    if a2 < 1e-8:
        a2 = 0.0
    elif a2 > (model.C - 1e-8):
        a2 = model.C
    if abs(a2 - alph2) < eps*(a2+alph2+eps):
        return 0, model
    
    a1 = alph1 + s*(alph2 - a2)
    b1 = E1 + y1 * (a1 - alph1) * k11 + y2 * (a2 - alph2) * k12 + model.b
    b2 = E2 + y1 * (a1 - alph1) * k12 + y2 * (a2 - alph2) * k22 + model.b
     
    if 0 < a1 and a1 < model.C:
        b = b1
    elif 0 < a2 and a2 < model.C:
        b = b2
    else:
        b = (b1 + b2) * 0.5
    
    model.alphas[i1] = a1
    model.alphas[i2] = a2

    for index, alph in zip([i1, i2], [a1, a2]):
        if 0.0 < alph and alph < model.C:
            model.errors[index] = 0.0
     
    NonOpt = list(filter(lambda n: n != i1 and n != i2, list(range(model.m))))
    model.errors[NonOpt] = model.errors[NonOpt] + y1*(a1 - alph1)*model.kernel(model.X[i1], model.X[NonOpt]) +y2*(a2 - alph2)*model.kernel(model.X[i2], model.X[NonOpt]) + model.b - b
    model.b = b
    
    return 1, model

In [ ]:
def examineExample(i2, model):
    
    y2 = model.y[i2]
    alph2 = model.alphas[i2]
    E2 = model.errors[i2]
    r2 = E2*y2
    
    if ((r2 < -tol and alph2 < model.C) or (r2 > tol and alph2 > 0)):
        if len(model.alphas[(model.alphas != 0) & (model.alphas != model.C)]) > 1:
            if model.errors[i2] > 0:
                i1 = np.argmin(model.errors)
            elif model.errors[i2] <= 0:
                i1 = np.argmax(model.errors)  
            step, model = takeStep(i1, i2, model)
            if step:
                return 1, model
         
        for i1 in np.roll(np.where((model.alphas != 0) & (model.alphas != model.C))[0], np.random.choice(np.arange(model.m))):
            step, model = takeStep(i1, i2, model)
            if step:
                return 1, model
        
        for i1 in np.roll(np.arange(model.m), np.random.choice(np.arange(model.m))):
            step, model = takeStep(i1, i2, model)
            if step:
                return 1, model
            
    return 0, model

In [ ]:
def train(model):
    numChanged = 0
    examineAll = 1

    while numChanged > 0 or examineAll:
        numChanged = 0
        if examineAll:
            for i in range(model.m):
                res , model = examineExample(i, model)
                numChanged += res
                
        else:
            for i in np.where((model.alphas != 0) & (model.alphas != model.C))[0]:
                res , model = examineExample(i, model)
                numChanged += res
                
        if examineAll == 1:
            examineAll = 0
        elif numChanged == 0:
            examineAll = 1
    return model

In [ ]:
def predict(model,X_test):
    result = np.zeros(X_test.shape[0])
    for i,test in enumerate(X_test):
        result[i] = decisionFunction(model.alphas, model.y, model.kernel, model.X, test, model.b)
        if result[i] > 0:
            result[i] = 1
        else:
            result[i] = -1
    return result

def Classifier(model0, model1, model2, X_test):
    result = np.zeros(X_test.shape[0])
    for i,test in enumerate(X_test):
        result[i] = np.argmax(np.array([
            predict(model0,np.array([test]))[0],
            predict(model1,np.array([test]))[0]+0.0001,
            predict(model2,np.array([test]))[0]+0.0001
        ]))
    return result

In [ ]:
xTrain, xTest, yTrain, yTest = train_test_split(dataDf.iloc[:,0:-4], dataDf['PASSED'], test_size=0.4, random_state=42)
xVal, xTest, yVal, yTest = train_test_split(xTest, yTest, test_size=0.5, random_state=42)
xTrain, xVal, xTest, yTrain, yTest, yVal = xTrain.to_numpy(), xVal.to_numpy(), xTest.to_numpy(), yTrain.to_numpy(), yTest.to_numpy(), yVal.to_numpy() 

model = SMOModel(xTrain, yTrain, 1000.0, 0, gaussianKernel)
np.random.seed(0)
model = train(model)

result = np.zeros(xTest.shape[0])
for i,test in enumerate(xTest):
    result[i] = np.argmax(np.array([predict(model,np.array([test]))[0]]))